In [114]:
!pip install gymnasium

In [115]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [199]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [156]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [157]:
env.observation_space.n

64

In [159]:
env.action_space.n

4

In [120]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [121]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 43)

In [122]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 7)

In [123]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Right)
SFFF
FHFH
FFFH
HFFG



(0.0, 9)

In [124]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Right)
SFFF
FHFH
FFFH
HFFG



(0.0, 11)

In [125]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 29)

In [126]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [127]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [128]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 44/1000
Average number of steps: 11.568181818181818


In [129]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 112/1000
Average number of steps: 16.133928571428573


In [130]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 781/1000
Average number of steps: 45.27400768245839


In [131]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [132]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [133]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [134]:
np.all(v_values_1 >= v_values_0)

True

In [135]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [136]:
np.all(v_values_2 >= v_values_1)

True

In [137]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [138]:
np.all(v_values_3 >= v_values_2)

True

In [184]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    start_time = time.time()
    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break

    end_time = time.time()
    print("Value Iteration run time: %.2f" % (end_time - start_time))
    return v_values

In [241]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    start_time = time.time()
    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    end_time = time.time()
    print("Policy Extraction run time: %.2f" % (end_time - start_time))
    return policy

In [183]:
def policy_iteration(env, max_iters=100, gamma=0.9):

    # Initialize a random policy (e.g., uniform random or deterministic)
    policy = np.full((env.observation_space.n, env.action_space.n), 1 / env.action_space.n)
    v_values = np.zeros(env.observation_space.n)

    start_time = time.time()
    for i in range(max_iters):
        # Policy evaluation
        while True:
            prev_v_values = np.copy(v_values)

            for state in range(env.observation_space.n):
                v_prev = v_values[state]
                q_values = []

                for action in range(env.action_space.n):
                    q_value = 0
                    for prob, next_state, reward, _ in env.P[state][action]:
                        q_value += prob * (reward + gamma * v_values[next_state])

                    q_values.append(policy[state][action] * q_value)

                v_values[state] = sum(q_values)

            # check convergence
            if np.all(np.isclose(v_values, prev_v_values)):
                # print(f'Converged at {i}-th iteration.')
                break

        # Policy improvement
        new_policy = np.zeros((env.observation_space.n, env.action_space.n))

        for state in range(env.observation_space.n):
            q_values = []

            for action in range(env.action_space.n):
                q_value = 0
                for prob, next_state, reward, _ in env.P[state][action]:
                    q_value += prob * (reward + gamma * v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            new_policy[state][best_action] = 1

        if np.all(policy == new_policy):
            print(f'Converged at {i}-th iteration.')
            break

        policy = new_policy

    result = np.zeros(env.observation_space.n, dtype=np.int32)
    for i in range(env.observation_space.n):
        for j in range(env.action_space.n):
            if policy[i][j] == 1:
              result[i] = j

    end_time = time.time()
    print("Policy Iteration run time: %.2f" % (end_time - start_time))
    return v_values, result


# **Games to run**

In [258]:
env = gym.make('FrozenLake8x8-v1', render_mode="ansi")

In [266]:
env = gym.make('Taxi-v3', render_mode="ansi")

In [250]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

# **Running value iteration**

In [267]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 116-th iteration.
Value Iteration run time: 6.71


In [268]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

Policy Extraction run time: 0.06


In [269]:
optimal_policy

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 3, 1, 3, 2, 3, 3, 3, 3,
       1, 1, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [270]:
optimal_v_values

array([ 89.47323891,  32.81971401,  55.26423891,  37.57755845,
         8.43222921,  32.81971401,   8.43222921,  15.28447953,
        32.81971401,  18.09386122,  55.26423891,  21.2154998 ,
        12.75594298,  18.09386122,  12.75594298,  37.57755845,
       100.52591945,  37.57755845,  62.51591945,  42.86394891,
        79.52591945,  28.53774704,  48.73781945,  32.81971401,
        10.48035311,  37.57755845,  10.48035311,  18.09386122,
        28.53774704,  15.28447953,  48.73781945,  18.09386122,
        15.28447953,  21.2154998 ,  15.28447953,  42.86394891,
        89.47323891,  42.86394891,  55.26423891,  48.73781945,
        42.86394891,  12.75594298,  24.68388374,  15.28447953,
        24.68388374,  70.57323891,  24.68388374,  37.57755845,
        24.68388374,  12.75594298,  42.86394891,  15.28447953,
        18.09386122,  24.68388374,  18.09386122,  48.73781945,
        48.73781945,  79.52591945,  48.73781945,  55.26423891,
        37.57755845,  10.48035311,  21.2154998 ,  12.75

In [271]:
v_values_optimal = policy_evaluation(env, optimal_policy)

Converged at 116-th iteration.


In [274]:
play(env, optimal_policy, True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



(9, 12)

In [273]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 1000/1000
Average number of steps: 13.086


# **Running policy iteration**

In [235]:
optimal_v_values2, optimal_policy2 = policy_iteration(env, max_iters=500, gamma=0.9)

Converged at 12-th iteration.
Policy Iteration run time: 8.29


In [236]:
optimal_policy2

array([4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 1, 2, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 1, 2, 3, 2, 3, 3,
       3, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3,
       2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 3, 2, 3, 0, 3, 3, 3, 3, 1, 1,
       1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 3, 1, 3, 0, 1,

In [237]:
optimal_v_values2

array([ 89.47368421,  32.82015931,  55.26468421,  37.57795479,
         8.43267451,  32.82015931,   8.43267451,  15.28487587,
        32.82015931,  18.09430652,  55.26468421,  21.21589614,
        12.75638828,  18.09430652,  12.75638828,  37.57795479,
       100.52631579,  37.57795479,  62.51631579,  42.86439421,
        79.52631579,  28.53814338,  48.73821579,  32.82015931,
        10.48074946,  37.57795479,  10.48074946,  18.09430652,
        28.53814338,  15.28487587,  48.73821579,  18.09430652,
        15.28487587,  21.21589614,  15.28487587,  42.86439421,
        89.47368421,  42.86439421,  55.26468421,  48.73821579,
        42.86439421,  12.75638828,  24.68432904,  15.28487587,
        24.68432904,  70.57368421,  24.68432904,  37.57795479,
        24.68432904,  12.75638828,  42.86439421,  15.28487587,
        18.09430652,  24.68432904,  18.09430652,  48.73821579,
        48.73821579,  79.52631579,  48.73821579,  55.26468421,
        37.57795479,  10.48074946,  21.21589614,  12.75

In [238]:
v_values_optimal2 = policy_evaluation(env, optimal_policy2)

Converged at 116-th iteration.


In [239]:
play(env, optimal_policy2, True)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



(11, 10)

In [240]:
play_multiple_times(env, optimal_policy2, 1000)

Number of successes: 1000/1000
Average number of steps: 13.007


# **Nhận xét**

**- FrozenLake-v1:**

value_iteration: 0.12s (iteration) + 0.00s (extraction) = 0.12s    

policy_iteration: 0.08s

**- FrozenLake8x8-v1:**

value_iteration: 0.71s + 0.01s = 0.72s    

policy_iteration: 0.64s

**- Taxi-v3:**

value_iteration: 6.53s + 0.06s = 6.59s    

policy_iteration: 8.29s

**TB value_iteration:** 2.48s

**TB policy_iteration:** 3.00s

Theo thời gian chạy trung bình, value_iteration cho ra kết quả nhanh hơn